# Dask Array

Материалы:
* Макрушин С.В. Лекция 11: Dask
* https://docs.dask.org/en/latest/array.html
* JESSE C. DANIEL. Data Science with Python and Dask. 

## Задачи для совместного разбора

1. Создайте массив размерностью 1000 на 300000, заполненный числами из стандартного нормального распределения. Исследуйте основные характеристики полученного массива.

2. Посчитайте сумму квадратов элементов массива, созданного в задаче 1. Создайте массив `np.array` такого же размера и сравните скорость решения задачи с использование `da.array` и `np.array`

3. Визуализируйте граф вычислений для задачи 12.

## Лабораторная работа 11

In [59]:
import dask.array as da
import h5py
import numpy as np

In [60]:
conda install -c conda-forge python-graphviz

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


1. Считайте датасет `recipe` из файла `minutes_n_ingredients_full.hdf5` в виде `dask.array`. Укажите аргумент `chunks=(100_000, 3)` при создании массива. Выведите на экран основную информацию о массиве.

In [61]:
f = h5py.File('minutes_n_ingredients_full.hdf5', 'r')
list(f.keys())

['recipe']

In [62]:
data_set = f['/recipe']

In [68]:
x_da = da.from_array(data_set, chunks=(100_000, 3)); x_da

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(100000, 3), chunktype=numpy.ndarray>

In [64]:
x_da.compute()

array([[ 683970,      33,       9],
       [1089012,      23,       5],
       [1428572,       0,       5],
       ...,
       [1910650,      60,       2],
       [ 713836,       0,       9],
       [ 660699,      64,       8]], dtype=int64)

2. Вычислите среднее значение по каждому столбцу, кроме первого. 

In [65]:
x_da.mean(axis=0).compute()[1:]

array([1004.20805176,    5.4198008 ])

3. Исследуйте, как влияет значение аргумента `chunks` при создании `dask.array` на скорость выполнения операции поиска среднего. 

In [80]:
x_da1 = da.from_array(data_set, chunks=(1000000, 3)); x_da1

dask.array<array, shape=(2231637, 3), dtype=int64, chunksize=(1000000, 3), chunktype=numpy.ndarray>

In [149]:
type(x_da1.mean(axis=0))

dask.array.core.Array

In [81]:
%%time
x_da1.mean(axis=0).compute()

Wall time: 37 ms


array([1.12684089e+06, 1.00420805e+03, 5.41980080e+00])

In [82]:
x_da2 = da.from_array(data_set, chunks=(1000, 3)); x_da2

,Array,Chunk
Bytes,53.56 MB,24.00 kB
Shape,"(2231637, 3)","(1000, 3)"
Count,2233 Tasks,2232 Chunks
Type,int64,numpy.ndarray


In [148]:
%%time
x_da2.mean(axis=0).compute()

Wall time: 880 ms


numpy.ndarray

In [85]:
# чем больше чанков мы создаем, тем дольше будет выполняться операция по выполнению среднего

4. Выберите рецепты, время выполнения которых меньше медианного значения

In [126]:
a = da.median(x_da, axis=0)[1].compute()

In [127]:
x_da[x_da[:,1]<a].compute()

array([[1089012,      23,       5],
       [1428572,       0,       5],
       [1400250,      24,       1],
       ...,
       [1029131,      19,       4],
       [1700703,       1,       1],
       [ 713836,       0,       9]], dtype=int64)

In [128]:
len(x_da[x_da[:,1]<a].compute())

1084304

5. Посчитайте количество каждого из возможных значений кол-ва ингредиентов

In [135]:
b = da.unique(x_da[:,2], return_counts = True)

In [136]:
b[0].compute()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 43], dtype=int64)

In [137]:
b[1].compute()

array([222071, 224158, 229388, 234948, 240720, 244360, 247181, 246747,
       246816,  22430,  19094,  15165,  11640,   8284,   6014,   4145,
         2793,   1913,   1279,    852,    529,    346,    244,    178,
          107,     68,     55,     33,     22,     20,     13,      5,
            4,      3,      4,      1,      2,      1,      1,      2,
            1], dtype=int64)

6. Найдите максимальную продолжительность рецепта. Ограничьте максимальную продолжительность рецептов сверху значением, равному 75% квантилю.

In [147]:
c = da.percentile(x_da[:, 1], 75).compute(); c

array([49.])

7. Создайте массив `dask.array` из 2 чисел, содержащих ваши предпочтения относительно времени выполнения рецепта и кол-ва ингредиентов. Найдите наиболее похожий (в смысле $L_1$) рецепт из имеющихся в датасете.

8. Работая с исходным файлом в формате `hdf5`, реализуйте алгоритм подсчета среднего значения в блочной форме и вычислите с его помощью среднее значение второго столбца в массиве.

Блочный алгоритм вычислений состоит из двух частей:
1. Загрузка фрагмента за фрагментом данных по `blocksize` элементов и проведение вычислений на этим фрагментом
2. Агрегация результатов вычислений на различных фрагментах для получения результата на уровне всего набора данных

Важно: при работе с `h5py` в память загружаются не все элементы, а только те, которые запрашиваются в данный момент